<a href="https://colab.research.google.com/github/ojuba-org/arabic-ml-data/blob/master/tashkeel_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensorflow documentation

* [v2](https://www.tensorflow.org/tutorials/text/text_generation)
* [v1](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/sequences/text_generation.ipynb)

In [1]:
%tensorflow_version 2.x

In [2]:
import sys
import os
import time

In [3]:
import re
import unicodedata

In [4]:
import numpy as np

In [5]:
import tensorflow as tf

In [6]:
#tf.enable_eager_execution()

In [7]:
#import tensorflow.compat.v1 as tf

In [8]:
print("tf version = {} and py version = {}".format(tf.__version__, sys.version))

tf version = 2.7.0 and py version = 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [9]:
assert sys.version_info.major == 3, 'please use python 3'
assert tf.test.gpu_device_name()!='', 'no GPU, please enable GPU'

In [10]:
#from tensorflow.keras.layers.experimental import preprocessing


In [11]:
! curl -sSLO https://github.com/ojuba-org/arabic-ml-data/archive/refs/heads/master.zip

In [12]:
! unzip -f -q master.zip "arabic-ml-data-master/corpora/quran/*.txt"

In [13]:
! cat arabic-ml-data-master/corpora/quran/*.txt > data.txt; wc -l data.txt

6236 data.txt


In [14]:
lines=[l.strip() for l in open('data.txt', 'r') if l.strip()]
s=set()
for l in lines: s.update(l)

In [15]:
alpha=sorted(s)

In [16]:
tashkeel_set = { ch for ch in alpha if unicodedata.category(ch)=='Mn' }

In [17]:
def tokenize_tashkeel(line):
  offs=[ ix for ix, ch in enumerate(line) if ch not in tashkeel_set ]
  offs.append(len(line))
  return [ line[off: offs[ix+1]] for ix, off in enumerate(offs[:-1]) ]

In [18]:
(
  VOC_START,
  VOC_UNKNOWN,
  VOC_UNKNOWN_TASHKEEL, VOC_UNKNOWN_SHADDA,
  VOC_UNKNOWN_LETTER, VOC_UNKNOWN_HAMZA, VOC_UNKNOWN_TEH_OR_HAA, VOC_UNKNOWN_MAQSORA,
  VOC_FATHATAN, VOC_DAMMATAN, VOC_KASRATAN, 
  VOC_FATHA, VOC_DAMMA, VOC_KASRA,
  VOC_SHADDA_FATHA, VOC_SHADDA_DAMMA, VOC_SHADDA_KASRA,
  VOC_SUKUN,
  VOC_STOP,
) = range(19)

In [19]:
chars = [ ch for ch in alpha if unicodedata.category(ch)!='Mn']

In [20]:
ch2id = { ch: ix+VOC_STOP+1 for ix, ch in enumerate(chars)}

In [21]:
ch2id.update({
  "": VOC_START,
  "\n": VOC_STOP,
  "\u064B": VOC_FATHATAN, "\u064C": VOC_DAMMATAN, "\u064D": VOC_KASRATAN,
  "\u064E": VOC_FATHA, "\u064F": VOC_DAMMA, "\u0650": VOC_KASRA,
  "\u0651\u064E": VOC_SHADDA_FATHA, "\u0651\u064F": VOC_SHADDA_DAMMA, "\u0651\u0650": VOC_SHADDA_KASRA,
  "\u0651": VOC_SUKUN,
  "\u061f": VOC_UNKNOWN,
  "\u0640\u061f\u0640": VOC_UNKNOWN_TASHKEEL,
  "\u0651\u061f": VOC_UNKNOWN_SHADDA,
})

In [22]:
id2ch = { v: k for k, v in ch2id.items() }

In [23]:
voc_size = max(ch2id.values())

In [24]:
id2ch.update({ ix:'\u061f' for ix in range(voc_size) if ix not in id2ch})

In [25]:
id_repr = { globals()[i]: i for i in dir() if i.startswith('VOC_')}
id_repr.update({i: unicodedata.name(id2ch[i]) for i in range(VOC_STOP+1, voc_size)})

In [26]:
def tokenize_one_id(sub):
  return (ch2id.get(sub[0], VOC_UNKNOWN),
          ch2id.get(sub[1:] or "\u0640\u061f\u0640", VOC_UNKNOWN_TASHKEEL),
  )
def tokenize_tashkeel_id(line):
  return [tokenize_one_id(sub) for sub in tokenize_tashkeel(line)]

In [27]:
for i in alpha: print(unicodedata.name(i), repr(i))

SPACE ' '
ARABIC LETTER HAMZA 'ء'
ARABIC LETTER ALEF WITH MADDA ABOVE 'آ'
ARABIC LETTER ALEF WITH HAMZA ABOVE 'أ'
ARABIC LETTER WAW WITH HAMZA ABOVE 'ؤ'
ARABIC LETTER ALEF WITH HAMZA BELOW 'إ'
ARABIC LETTER YEH WITH HAMZA ABOVE 'ئ'
ARABIC LETTER ALEF 'ا'
ARABIC LETTER BEH 'ب'
ARABIC LETTER TEH MARBUTA 'ة'
ARABIC LETTER TEH 'ت'
ARABIC LETTER THEH 'ث'
ARABIC LETTER JEEM 'ج'
ARABIC LETTER HAH 'ح'
ARABIC LETTER KHAH 'خ'
ARABIC LETTER DAL 'د'
ARABIC LETTER THAL 'ذ'
ARABIC LETTER REH 'ر'
ARABIC LETTER ZAIN 'ز'
ARABIC LETTER SEEN 'س'
ARABIC LETTER SHEEN 'ش'
ARABIC LETTER SAD 'ص'
ARABIC LETTER DAD 'ض'
ARABIC LETTER TAH 'ط'
ARABIC LETTER ZAH 'ظ'
ARABIC LETTER AIN 'ع'
ARABIC LETTER GHAIN 'غ'
ARABIC LETTER FEH 'ف'
ARABIC LETTER QAF 'ق'
ARABIC LETTER KAF 'ك'
ARABIC LETTER LAM 'ل'
ARABIC LETTER MEEM 'م'
ARABIC LETTER NOON 'ن'
ARABIC LETTER HEH 'ه'
ARABIC LETTER WAW 'و'
ARABIC LETTER ALEF MAKSURA 'ى'
ARABIC LETTER YEH 'ي'
ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
A

In [28]:
for i in sorted(tashkeel_set): print(unicodedata.name(i), repr(i))

ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
ARABIC DAMMA 'ُ'
ARABIC KASRA 'ِ'
ARABIC SHADDA 'ّ'
ARABIC SUKUN 'ْ'


In [29]:
line=lines[0]; line

'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ'

In [30]:
print(len(line))
tokens=tokenize_tashkeel(line)
print(tokens)
[ len(token) for token in tokens]

37
['بِ', 'سْ', 'مِ', ' ', 'ا', 'ل', 'لَّ', 'هِ', ' ', 'ا', 'ل', 'رَّ', 'حْ', 'مَ', 'نِ', ' ', 'ا', 'ل', 'رَّ', 'حِ', 'ي', 'مِ']


[2, 2, 2, 1, 1, 1, 3, 2, 1, 1, 1, 3, 2, 2, 2, 1, 1, 1, 3, 2, 1, 2]

In [31]:
VOC_STOP

18

In [32]:
print(voc_size)

55


In [33]:
id2ch

{0: '',
 1: '؟',
 2: 'ـ؟ـ',
 3: 'ّ؟',
 4: '؟',
 5: '؟',
 6: '؟',
 7: '؟',
 8: 'ً',
 9: 'ٌ',
 10: 'ٍ',
 11: 'َ',
 12: 'ُ',
 13: 'ِ',
 14: 'َّ',
 15: 'ُّ',
 16: 'ِّ',
 17: 'ّ',
 18: '\n',
 19: ' ',
 20: 'ء',
 21: 'آ',
 22: 'أ',
 23: 'ؤ',
 24: 'إ',
 25: 'ئ',
 26: 'ا',
 27: 'ب',
 28: 'ة',
 29: 'ت',
 30: 'ث',
 31: 'ج',
 32: 'ح',
 33: 'خ',
 34: 'د',
 35: 'ذ',
 36: 'ر',
 37: 'ز',
 38: 'س',
 39: 'ش',
 40: 'ص',
 41: 'ض',
 42: 'ط',
 43: 'ظ',
 44: 'ع',
 45: 'غ',
 46: 'ف',
 47: 'ق',
 48: 'ك',
 49: 'ل',
 50: 'م',
 51: 'ن',
 52: 'ه',
 53: 'و',
 54: 'ى',
 55: 'ي'}

In [34]:
id_repr

{0: 'VOC_START',
 1: 'VOC_UNKNOWN',
 2: 'VOC_UNKNOWN_TASHKEEL',
 3: 'VOC_UNKNOWN_SHADDA',
 4: 'VOC_UNKNOWN_LETTER',
 5: 'VOC_UNKNOWN_HAMZA',
 6: 'VOC_UNKNOWN_TEH_OR_HAA',
 7: 'VOC_UNKNOWN_MAQSORA',
 8: 'VOC_FATHATAN',
 9: 'VOC_DAMMATAN',
 10: 'VOC_KASRATAN',
 11: 'VOC_FATHA',
 12: 'VOC_DAMMA',
 13: 'VOC_KASRA',
 14: 'VOC_SHADDA_FATHA',
 15: 'VOC_SHADDA_DAMMA',
 16: 'VOC_SHADDA_KASRA',
 17: 'VOC_SUKUN',
 18: 'VOC_STOP',
 19: 'SPACE',
 20: 'ARABIC LETTER HAMZA',
 21: 'ARABIC LETTER ALEF WITH MADDA ABOVE',
 22: 'ARABIC LETTER ALEF WITH HAMZA ABOVE',
 23: 'ARABIC LETTER WAW WITH HAMZA ABOVE',
 24: 'ARABIC LETTER ALEF WITH HAMZA BELOW',
 25: 'ARABIC LETTER YEH WITH HAMZA ABOVE',
 26: 'ARABIC LETTER ALEF',
 27: 'ARABIC LETTER BEH',
 28: 'ARABIC LETTER TEH MARBUTA',
 29: 'ARABIC LETTER TEH',
 30: 'ARABIC LETTER THEH',
 31: 'ARABIC LETTER JEEM',
 32: 'ARABIC LETTER HAH',
 33: 'ARABIC LETTER KHAH',
 34: 'ARABIC LETTER DAL',
 35: 'ARABIC LETTER THAL',
 36: 'ARABIC LETTER REH',
 37: 'ARABIC LETTE

In [35]:
tokens=tokenize_tashkeel_id(line)
for con_id, vo_id in tokens: print(con_id, id_repr.get(con_id), ", ", vo_id, id_repr.get(vo_id))

27 ARABIC LETTER BEH ,  13 VOC_KASRA
38 ARABIC LETTER SEEN ,  2 VOC_UNKNOWN_TASHKEEL
50 ARABIC LETTER MEEM ,  13 VOC_KASRA
19 SPACE ,  2 VOC_UNKNOWN_TASHKEEL
26 ARABIC LETTER ALEF ,  2 VOC_UNKNOWN_TASHKEEL
49 ARABIC LETTER LAM ,  2 VOC_UNKNOWN_TASHKEEL
49 ARABIC LETTER LAM ,  14 VOC_SHADDA_FATHA
52 ARABIC LETTER HEH ,  13 VOC_KASRA
19 SPACE ,  2 VOC_UNKNOWN_TASHKEEL
26 ARABIC LETTER ALEF ,  2 VOC_UNKNOWN_TASHKEEL
49 ARABIC LETTER LAM ,  2 VOC_UNKNOWN_TASHKEEL
36 ARABIC LETTER REH ,  14 VOC_SHADDA_FATHA
32 ARABIC LETTER HAH ,  2 VOC_UNKNOWN_TASHKEEL
50 ARABIC LETTER MEEM ,  11 VOC_FATHA
51 ARABIC LETTER NOON ,  13 VOC_KASRA
19 SPACE ,  2 VOC_UNKNOWN_TASHKEEL
26 ARABIC LETTER ALEF ,  2 VOC_UNKNOWN_TASHKEEL
49 ARABIC LETTER LAM ,  2 VOC_UNKNOWN_TASHKEEL
36 ARABIC LETTER REH ,  14 VOC_SHADDA_FATHA
32 ARABIC LETTER HAH ,  13 VOC_KASRA
55 None ,  2 VOC_UNKNOWN_TASHKEEL
50 ARABIC LETTER MEEM ,  13 VOC_KASRA


In [ ]:
import functools

In [ ]:
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
